In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from python_crud_module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "WaffleMaker1"
host = "nv-desktop-services.apporto.com"
port = "31669"
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(f"mongodb://{username}:{password}@{host}:{port}")

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load and encode logo in base64
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(style={'display': 'flex', 'align-items': 'center', 'justify-content':'center'}, children=[
        html.A([
           html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width':'150px', 'height':'150px'})
           ], href="https://www.snhu.edu", target="_blank"),
        html.Div(style={'display':'flex','flex-direction':'column', 'align-items':'center', 'justify-content':'center'}, children=[
            html.B(html.H1("Project Two - Lance Cain")),
            html.B(html.H1('CS-340 Dashboard'))
        ])
        
    ]),
    #html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width':'150px', 'height':'150px'})),
    #html.Center(html.B(html.H1("Project Two - Lance Cain"))),
    #html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'MW'},
                {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DRIT'},
                {'label': 'Reset - returns unfiltered state', 'value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block'}
        )
    ),

    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):

    if filter_type == 'WR':
        df = pd.DataFrame.from_records(db.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                {'$or': [
                                                    {'breed': 'Labrador Retriever Mix'},
                                                    {'breed': 'Chesa Bay Retr Mix'},
                                                    {'breed': 'Newfoundland Mix'}]
                                                },
                                                {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                         {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                }]
                                       }))
    elif filter_type == 'MW':
            df = pd.DataFrame.from_records(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                {'$or': [
                                                    {'breed': 'German Sheperd'},
                                                    {'breed': 'Alaskan Malamute'},
                                                    {'breed': 'Old English Sheepdog'},
                                                    {'breed': 'Rottweiler'},
                                                    {'breed': 'Siberian Husky'}]
                                                },
                                                {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                         {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                }]
                                       }))
    elif filter_type == 'DRIT':
            df = pd.DataFrame.from_records(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                {'$or': [
                                                    {'breed': 'Doberman Pinsch'},
                                                    {'breed': 'German Shepherd'},
                                                    {'breed': 'Golden Retriever'},
                                                    {'breed': 'Bloodhound'},
                                                    {'breed': 'Rottweiler'}]
                                                },
                                                {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                         {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                }]
                                       }))
    elif filter_type == 'RESET':
            df = pd.DataFrame.from_records(db.read({}))
    else:
        df = pd.DataFrame.from_records(db.read({}))
    
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True)
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    return (data)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):

    dff = pd.DataFrame.from_dict(viewData)
    if 'breed' not in dff.columns:
        return dash.no_update
    
    breed_counts = dff['breed'].value_counts()
    
    #low_counts = breed_counts[breed_counts < 71].index
    #high_counts = breed_counts[breed_counts > 70]
    
    if len(breed_counts) > 30:
        low_counts_index = breed_counts[breed_counts < 71].index
        dff.loc[dff['breed'].isin(low_counts_index), 'breed'] = "Other"
    
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    
    return [
        dcc.Graph(
            figure = px.pie(
                data_frame=dff,
                values=values,
                names=names,
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800,
                height=500
                )
            )
        ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True, port=8051)


Dash app running on http://127.0.0.1:8051/
